In [1]:
# Load some of the packages
import numpy as np

# Generate the innovations of the time series (an MxT)
M, T = 3, 100
procs = np.random.normal(size=(M,T))

# Introduce first-order AR dependence on the innovations
def make_ar(x):
    for i in range(1,len(x)):
        x[i] += 0.5 * x[i-1]

make_ar(procs[0])
for i in range(1,M):
    procs[i][1:] += 0.25 * procs[i-1][:-1] # Add a time-lagged dependence onto the observations
    make_ar(procs[i])

In [12]:
# Load the Data class
from pynats.data import Data

# Create an unnamed dataset with 3 processes and 1000 observations (1 replication)
#   - dim_order specifies processes is the first dimension and samples/observations are the second
#   - normalise z-scores the data
data = Data(procs, dim_order='ps', normalise=True)

import pynats.plot as natplt

natplt.plot_spacetime(data)

In [3]:
# Load the Calculator class
from pynats.calculator import Calculator

# Let's create an unnamed calculator
calc = Calculator(dataset=data)

Loading configuration file: /home/oliver/Dropbox/Workspace/code/research/pynats/pynats/config.yaml
*** Importing module .correlation
[0] Adding measure .correlation.pearsonr(x,y,{'cov_estimator': 'empirical'})...
Succesfully initialised with identifier "pearsonr_empirical"
[1] Adding measure .correlation.pearsonr(x,y,{'cov_estimator': 'empirical', 'squared': True})...
Succesfully initialised with identifier "pearsonr-sq_empirical"
[2] Adding measure .correlation.pearsonr(x,y,{'cov_estimator': 'ledoit_wolf'})...
Succesfully initialised with identifier "pearsonr_ledoit_wolf"
[3] Adding measure .correlation.pearsonr(x,y,{'cov_estimator': 'shrunk'})...
Succesfully initialised with identifier "pearsonr_shrunk"
[4] Adding measure .correlation.pearsonr(x,y,{'cov_estimator': 'oas'})...
Succesfully initialised with identifier "pearsonr_oas"
[5] Adding measure .correlation.pcor(x,y,{'cov_estimator': 'empirical'})...
Succesfully initialised with identifier "pcorr_empirical"
[6] Adding measure .co

In [4]:
calc.compute()

Processing [None: te_gaussian_k-1_kt-1_l-1_lt-1]: 100%|██████████| 86/86 [00:16<00:00,  5.09it/s]


In [5]:
# Should be an adjacency matrix of features x processes x processes
calc.adjacency.shape

(86, 3, 3)

In [8]:
import pynats.plot as natplt

natplt.clustermap(calc)

(                      pearsonr_empirical  pearsonr-sq_em...cal  \
 pearsonr_empirical              1.000000              0.500000   
 pearsonr-sq_em...cal            0.500000              1.000000   
 pearsonr_ledoit_wolf            0.000000              0.000000   
 pearsonr_shrunk                 1.000000              0.500000   
 pearsonr_oas                    0.000000              0.000000   
 ...                                  ...                   ...   
 te_kraskov_NN-...t-1           -0.478091              0.478091   
 te_kraskov_NN-...t-1           -0.478091              0.478091   
 te_kernel_W-0.25_k-1            0.239046             -0.597614   
 te_gaussian_k-...x-2            0.836660              0.597614   
 te_gaussian_k-...t-1            0.836660              0.597614   
 
                       pearsonr_ledoit_wolf  pearsonr_shrunk  pearsonr_oas  \
 pearsonr_empirical                     0.0         1.000000           0.0   
 pearsonr-sq_em...cal                 